In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from utils import watershed_image, cut_cells, make_data
import random
from glob import glob
import shutil

# Download patient info including id, label and cell rate

In [4]:
img_dir = '../o_test/'
save_dir = './segmentation_results/npy/'
mat_dir = './segmentation_results/mat/'
patient_info_path = './patient_info.csv'
patient_id = np.array(pd.read_csv(patient_info_path,usecols=['patient id'],squeeze=True),dtype='str')
patient_label = np.array(pd.read_csv(patient_info_path,usecols=['label'],squeeze=True),dtype='int')
cell_rate = np.array(pd.read_csv(patient_info_path,usecols=['cell rate'],squeeze=True),dtype='float')

# Make data (a npy file per patient)

In [ ]:
patient_num = len(patient_id)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
for i in range(patient_num):
    patient_data = make_data(img_dir, mat_dir, patient_id[i], patient_label[i], cell_rate[i])
    np.save(save_dir + patient_id[i] + '.npy', patient_data)

# Make sampling ( n cells per sample)

In [ ]:
npy_dir = './segmentation_results/npy/'
sampling_save_dir = './segmentation_results/sampling/'
if not os.path.exists(sampling_save_dir):
    os.mkdir(sampling_save_dir)
sample_num = 50 # define the number of cells per sample
for patient in patient_id:
    i = 1
    npy_name = npy_dir + patient + '.npy'
    data = np.load(npy_name, allow_pickle=True)[()]
    p_label = data['p_label']
    c_rate = data['c_rate']
    cells = data['c_images']
    cells_insts = data['c_insts']
    num_cells = len(cells)
    original_list = list(range(0,num_cells))
    all_ = original_list.copy()
    while num_cells>0:
        sub_npy_name = sampling_save_dir + patient + '_' + str(i) + '.npy'
        i = i+1
        sub_patient = {}
        sub_cells = []
        sub_cells_insts = []

        if num_cells<sample_num:
            for idx in all_:
                sub_cells.append(cells[idx])
                sub_cells_insts.append(cells_insts[idx])
            while len(sub_cells)<sample_num:
                r = random.choice(original_list)
                sub_cells.append(cells[r])
                sub_cells_insts.append(cells_insts[r])
        else:
            choices = random.sample(all_, sample_num)
            for c in choices:
                sub_cells.append(cells[c])
                sub_cells_insts.append(cells_insts[c])
                all_.remove(c)

        sub_patient['p_label'] = p_label
        sub_patient['c_rate'] = c_rate
        sub_patient['c_images'] = sub_cells
        sub_patient['c_insts'] = sub_cells_insts
        np.save(sub_npy_name, sub_patient)
        num_cells = num_cells - sample_num

# Divide samples into k folds

In [ ]:
patient_num = len(patient_id)
disease_12=[[] for i in range(12)]
for i in range(patient_num):
    disease_12[patient_label[i]].append(patient_id[i])

k = 5
sample_dir = './segmentation_results/sampling/'
save_dir = './dataset/'
for i in disease_12:
    num = len(i)
    each = num//k
    original_list = list(range(0,num))
    all_ = original_list.copy()
    for j in range(k):
        dst_dir = save_dir + 'fold{}/'.format(j+1)
        if not os.path.exists(dst_dir):
            os.mkdir(dst_dir)
        if j == k-1:
            for d in all_:
                npys = glob(sample_dir + i[d] + '*npy')
                for npy in npys:
                    shutil.copy(npy, npy.replace(sample_dir, dst_dir))
        else:
            choices = random.sample(all_, each)
            for c in choices:
                npys = glob(sample_dir + i[c] + '*npy')
                for npy in npys:
                    shutil.copy(npy, npy.replace(sample_dir, dst_dir))
                all_.remove(c)